In [1]:
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
import adabelief_pytorch
global_checkpoint_period=np.inf

def GAN_MNIST(SA):
    import ray

    import os
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.utils.data
    import torchvision.datasets as dset
    import torchvision.transforms as transforms
    import torchvision.utils as vutils
    import numpy as np

    import ray
    from ray import tune
    from ray.tune.trial import ExportFormat
    from ray.tune.schedulers import PopulationBasedTraining

    import argparse
    import os
    from filelock import FileLock
    import random
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.optim as optim
    import torch.utils.data
    import numpy as np
    from ray.tune.suggest.bayesopt import BayesOptSearch
    from ray.tune.suggest.ax import AxSearch



    from torch.autograd import Variable
    from torch.nn import functional as F
    from scipy.stats import entropy

    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    # Training parameters
    dataroot = ray.utils.get_user_temp_dir() + os.sep
    workers = 2
    batch_size = 64
    image_size = 32

    # Number of channels in the training images. For color images this is 3
    nc = 1

    # Size of z latent vector (i.e. size of generator input)
    nz = 100

    # Size of feature maps in generator
    ngf = 32

    # Size of feature maps in discriminator
    ndf = 32

    # Beta1 hyperparam for Adam optimizers
    beta1 = 0.5

    # iterations of actual training in each Trainable _train
    train_iterations_per_step = 5

    MODEL_PATH = os.path.expanduser("~/.ray/models/mnist_cnn.pt")


    def get_data_loader():
        dataset = dset.MNIST(
            root=dataroot,
            download=True,
            transform=transforms.Compose([
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, ), (0.5, )),
            ]))

        # Create the dataloader
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

        return dataloader


    # __GANmodel_begin__
    # custom weights initialization called on netG and netD
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find("Conv") != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find("BatchNorm") != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)


    # Generator Code
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            self.main = nn.Sequential(
                # input is Z, going into a convolution
                nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
                nn.Tanh())

        def forward(self, input):
            return self.main(input)


    class Discriminator(nn.Module):
        def __init__(self):
            super(Discriminator, self).__init__()
            self.main = nn.Sequential(
                nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 2), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 4), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False), nn.Sigmoid())

        def forward(self, input):
            return self.main(input)


    # __GANmodel_end__


    # __INCEPTION_SCORE_begin__
    class Net(nn.Module):
        """
        LeNet for MNist classification, used for inception_score
        """

        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
            self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(320, 50)
            self.fc2 = nn.Linear(50, 10)

        def forward(self, x):
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)


    def inception_score(imgs, mnist_model_ref, batch_size=32, splits=1):
        N = len(imgs)
        dtype = torch.FloatTensor
        dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
        cm = ray.get(mnist_model_ref)  # Get the mnist model from Ray object store.
        up = nn.Upsample(size=(28, 28), mode="bilinear").type(dtype)

        def get_pred(x):
            x = up(x)
            x = cm(x)
            return F.softmax(x).data.cpu().numpy()

        preds = np.zeros((N, 10))
        for i, batch in enumerate(dataloader, 0):
            batch = batch.type(dtype)
            batchv = Variable(batch)
            batch_size_i = batch.size()[0]
            preds[i * batch_size:i * batch_size + batch_size_i] = get_pred(batchv)

        # Now compute the mean kl-div
        split_scores = []
        for k in range(splits):
            part = preds[k * (N // splits):(k + 1) * (N // splits), :]
            py = np.mean(part, axis=0)
            scores = []
            for i in range(part.shape[0]):
                pyx = part[i, :]
                scores.append(entropy(pyx, py))
            split_scores.append(np.exp(np.mean(scores)))

        return np.mean(split_scores), np.std(split_scores)


    # __INCEPTION_SCORE_end__


    def train(netD, netG, optimG, optimD, criterion, dataloader, iteration, device,
              mnist_model_ref):
        real_label = 1
        fake_label = 0

        for i, data in enumerate(dataloader, 0):
            if i >= train_iterations_per_step:
                break

            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full(
                (b_size, ), real_label, dtype=torch.float, device=device)
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimD.step()

            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimG.step()

            is_score, is_std = inception_score(fake, mnist_model_ref)

            # Output training stats
           # if iteration % 10 == 0:
            #    print("[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z))"
             #         ": %.4f / %.4f \tInception score: %.4f" %
              #        (iteration, len(dataloader), errD.item(), errG.item(), D_x,
               #        D_G_z1, D_G_z2, is_score))

        return errG.item(), errD.item(), is_score


    def plot_images(dataloader):
        # Plot some training images
        if 1==0:
            real_batch = next(iter(dataloader))
            plt.figure(figsize=(8, 8))
            plt.axis("off")
            plt.title("Original Images")
            plt.imshow(
                np.transpose(
                    vutils.make_grid(real_batch[0][:64], padding=2,
                                     normalize=True).cpu(), (1, 2, 0)))

            plt.show()


    def demo_gan(checkpoint_paths):
        img_list = []
        fixed_noise = torch.randn(64, nz, 1, 1)
        for netG_path in checkpoint_paths:
            loadedG = Generator()
            loadedG.load_state_dict(torch.load(netG_path)["netGmodel"])
            with torch.no_grad():
                fake = loadedG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        fig = plt.figure(figsize=(8, 8))
        plt.axis("off")
        ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)]
               for i in img_list]
        ani = animation.ArtistAnimation(
            fig, ims, interval=1000, repeat_delay=1000, blit=True)
        ani.save("./generated" +str(SA)+"best.gif", writer="imagemagick", dpi=72)
        plt.show()




    # __Trainable_begin__
    class PytorchTrainable(tune.Trainable):
        def setup(self, config):
            use_cuda = config.get("use_gpu") and torch.cuda.is_available()
            self.device = torch.device("cuda" if use_cuda else "cpu")
            self.netD = Discriminator().to(self.device)
            self.netD.apply(weights_init)
            self.netG = Generator().to(self.device)
            self.netG.apply(weights_init)
            self.criterion = nn.BCELoss()    
            self.optimizerD =optim.Adam(
                self.netD.parameters(),
                lr=config.get("lr", 0.01),
                betas=(beta1, 0.999))
            self.optimizerG =optim.Adam(
                self.netG.parameters(),
                lr=config.get("lr", 0.01),
                betas=(beta1, 0.999))

            with FileLock(os.path.expanduser("~/.data.lock")):
                self.dataloader = get_data_loader()
            self.mnist_model_ref = c["mnist_model_ref"]

        def step(self):
            lossG, lossD, is_score = train(self.netD, self.netG, self.optimizerG,
                                           self.optimizerD, self.criterion,
                                           self.dataloader, self._iteration,
                                           self.device, self.mnist_model_ref)
            return {"lossg": lossG, "lossd": lossD, "is_score": is_score}

        def save_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save({
                "netDmodel": self.netD.state_dict(),
                "netGmodel": self.netG.state_dict(),
                "optimD": self.optimizerD.state_dict(),
                "optimG": self.optimizerG.state_dict(),
            }, path)

            return checkpoint_dir

        def load_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            checkpoint = torch.load(path)
            self.netD.load_state_dict(checkpoint["netDmodel"])
            self.netG.load_state_dict(checkpoint["netGmodel"])
            self.optimizerD.load_state_dict(checkpoint["optimD"])
            self.optimizerG.load_state_dict(checkpoint["optimG"])

        def reset_config(self, new_config):
            if "netD_lr" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["lr"] = 10**-(new_config["netD_lr"])
            if "netG_lr" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["lr"] = 10**-(new_config["netG_lr"])
            if "netD_B" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["betas"] = (10**-(new_config["netD_B"]),0.999)
            if "netG_B" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["betas"] = (10**-(new_config["netG_B"]),0.999)
            if "weight_decay1" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay1"])
            if "weight_decay2" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay2"])
            self.config = new_config
            return True

        def _export_model(self, export_formats, export_dir):
            if export_formats == [ExportFormat.MODEL]:
                path = os.path.join(export_dir, "exported_models")
                torch.save({
                    "netDmodel": self.netD.state_dict(),
                    "netGmodel": self.netG.state_dict()
                }, path)
                return {ExportFormat.MODEL: path}
            else:
                raise ValueError("unexpected formats: " + str(export_formats))



    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    import urllib.request
    # Download a pre-trained MNIST model for inception score calculation.
    # This is a tiny model (<100kb).
    if not os.path.exists(MODEL_PATH):
        print("downloading model")
        os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
        urllib.request.urlretrieve(
            "https://github.com/ray-project/ray/raw/master/python/ray/tune/"
            "examples/pbt_dcgan_mnist/mnist_cnn.pt", MODEL_PATH)

    dataloader = get_data_loader()
    if not args.smoke_test:
        plot_images(dataloader)

    # load the pretrained mnist classification model for inception_score
    mnist_cnn = Net()
    mnist_cnn.load_state_dict(torch.load(MODEL_PATH))
    mnist_cnn.eval()
    mnist_model_ref = ray.put(mnist_cnn)

    # __tune_begin__
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=5,
        hyperparam_mutations={
            # distribution for resampling
            "netG_lr": lambda: np.random.uniform(1, 9),
            "netD_lr": lambda: np.random.uniform(1, 9),
            "netD_B": lambda: np.random.uniform(0, 2),
            "netD_B": lambda: np.random.uniform(0, 2),
            "weight_decay1":lambda: np.random.uniform(3, 8),
            "weight_decay2":lambda: np.random.uniform(3, 8)
        })
    from ray.tune.schedulers.pb2 import PB2

    scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=5,
    hyperparam_bounds={
        # distribution for resampling
            "netG_lr": [1, 9],
            "netD_lr": [1, 9],
            "netD_B": [0, 2],
            "netD_B": [0, 2],
            "weight_decay1":[3, 8],
            "weight_decay2":[3, 8]
    }) 
    
    c={"mnist_model_ref" : mnist_model_ref}


    experiment_metrics= dict(metric="is_score",
        mode="max")

 
    
    if(SA==0):
        algo = HyperOptSearch(**experiment_metrics)
        name="hypher"

    if(SA==1):
        algo =   BayesOptSearch(**experiment_metrics) 
        name="Bayes"
        
    if(SA==2):
        algo = AxSearch(
            max_concurrent=2, #was working with 2
            **experiment_metrics
        )
        name="AX"

        
        
    if(SA==3):
        algo = NevergradSearch(
            optimizer=ng.optimizers.CMA,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG"

        
     
    if(SA==4):
        algo = NevergradSearch(
            optimizer=ng.optimizers.TwoPointsDE,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG2DE"

        
        
    if(SA==5):
        algo = NevergradSearch(
        optimizer=ng.optimizers.RandomSearch,**experiment_metrics
        # space=space,  # If you want to set the space manually
        )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="random"
                
    if(SA==6):
        dim_dict = {
        "netG_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2),
        "netD_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2)
        }
        algo = ZOOptSearch(
        algo="Asracos",  # only support Asracos currently
        #dim_dict=dim_dict,
      #  dim_dict=dim_dict,

        budget=100,
            **experiment_metrics
        #dim_dict=dim_dict,
       #     **zoopt_search_config,
        )
        name="zoo"


  #  if(SA!=4):
    #scheduler = MedianStoppingRule()
    #AsyncHyperBandScheduler()
        
        
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

    tune_kwargs = {
    "num_samples": 20 if args.smoke_test else 20,
      "stop":{
            "training_iteration": 100,
        },  

    "config": {
     "lr1":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    , "lr2":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    ,     "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0    
    ,     "beta1":2 #tune.uniform(1, 5),#,1e-4), #*10 et 0
     ,    "beta2":1 #tune.uniform(1, 5),#,1e-4), #*10 et 0
      ,  "adam1":0#tune.uniform(0,1),
      ,  "adam2":0#tune.uniform(0,1),
        }
    }
    def trial_name_id(trial):
        return f"{trial.trainable_name}_{trial.trial_id}"

    analysis = tune.run(
    PytorchTrainable,
    name=name,
    scheduler=scheduler,
    reuse_actors=True,
    search_alg=algo,
    verbose=2,
    checkpoint_at_end=True,
    num_samples=16,
   # export_formats=[ExportFormat.MODEL],
        config =     {
            "netG_lr": tune.uniform(1, 9),
           "netD_lr": tune.uniform(1, 9),
            "netG_B": tune.uniform(0, 2),
           "netD_B": tune.uniform(0, 2),
            "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)
        },      stop={
            "training_iteration": 100,
        },        metric="is_score",
        mode="max"
             ,     loggers=[TestLogger])

    all_trials = analysis.trials
    checkpoint_paths = [
        os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
        for t in all_trials
    ]
    demo_gan(checkpoint_paths)
    

/home/antoine/anaconda3/lib/python3.7/site-packages/dragonfly/utils/oper_utils.py:30: UserWarning:

cannot import name 'direct' from 'dragonfly.utils.direct_fortran' (/home/antoine/anaconda3/lib/python3.7/site-packages/dragonfly/utils/direct_fortran/__init__.py)
Could not import Fortran direct library. Dragonfly can still be used, but might be slightly slower. To get rid of this warning, install a numpy compatible Fortran compiler (e.g. gfortran) and the python-dev package and reinstall Dragonfly.



In [2]:
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
import adabelief_pytorch
global_checkpoint_period=np.inf

def GAN_MNIST_ZO(SA):
    import ray

    import os
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.utils.data
    import torchvision.datasets as dset
    import torchvision.transforms as transforms
    import torchvision.utils as vutils
    import numpy as np

    import ray
    from ray import tune
    from ray.tune.trial import ExportFormat
    from ray.tune.schedulers import PopulationBasedTraining

    import argparse
    import os
    from filelock import FileLock
    import random
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.optim as optim
    import torch.utils.data
    import numpy as np
    from ray.tune.suggest.bayesopt import BayesOptSearch
    from ray.tune.suggest.ax import AxSearch



    from torch.autograd import Variable
    from torch.nn import functional as F
    from scipy.stats import entropy

    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    # Training parameters
    dataroot = ray.utils.get_user_temp_dir() + os.sep
    workers = 2
    batch_size = 64
    image_size = 32

    # Number of channels in the training images. For color images this is 3
    nc = 1

    # Size of z latent vector (i.e. size of generator input)
    nz = 100

    # Size of feature maps in generator
    ngf = 32

    # Size of feature maps in discriminator
    ndf = 32

    # Beta1 hyperparam for Adam optimizers
    beta1 = 0.5

    # iterations of actual training in each Trainable _train
    train_iterations_per_step = 5

    MODEL_PATH = os.path.expanduser("~/.ray/models/mnist_cnn.pt")


    def get_data_loader():
        dataset = dset.MNIST(
            root=dataroot,
            download=True,
            transform=transforms.Compose([
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, ), (0.5, )),
            ]))

        # Create the dataloader
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

        return dataloader


    # __GANmodel_begin__
    # custom weights initialization called on netG and netD
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find("Conv") != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find("BatchNorm") != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)


    # Generator Code
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            self.main = nn.Sequential(
                # input is Z, going into a convolution
                nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
                nn.Tanh())

        def forward(self, input):
            return self.main(input)


    class Discriminator(nn.Module):
        def __init__(self):
            super(Discriminator, self).__init__()
            self.main = nn.Sequential(
                nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 2), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 4), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False), nn.Sigmoid())

        def forward(self, input):
            return self.main(input)


    # __GANmodel_end__


    # __INCEPTION_SCORE_begin__
    class Net(nn.Module):
        """
        LeNet for MNist classification, used for inception_score
        """

        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
            self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(320, 50)
            self.fc2 = nn.Linear(50, 10)

        def forward(self, x):
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)


    def inception_score(imgs, mnist_model_ref, batch_size=32, splits=1):
        N = len(imgs)
        dtype = torch.FloatTensor
        dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
        cm = ray.get(mnist_model_ref)  # Get the mnist model from Ray object store.
        up = nn.Upsample(size=(28, 28), mode="bilinear").type(dtype)

        def get_pred(x):
            x = up(x)
            x = cm(x)
            return F.softmax(x).data.cpu().numpy()

        preds = np.zeros((N, 10))
        for i, batch in enumerate(dataloader, 0):
            batch = batch.type(dtype)
            batchv = Variable(batch)
            batch_size_i = batch.size()[0]
            preds[i * batch_size:i * batch_size + batch_size_i] = get_pred(batchv)

        # Now compute the mean kl-div
        split_scores = []
        for k in range(splits):
            part = preds[k * (N // splits):(k + 1) * (N // splits), :]
            py = np.mean(part, axis=0)
            scores = []
            for i in range(part.shape[0]):
                pyx = part[i, :]
                scores.append(entropy(pyx, py))
            split_scores.append(np.exp(np.mean(scores)))

        return np.mean(split_scores), np.std(split_scores)


    # __INCEPTION_SCORE_end__


    def train(netD, netG, optimG, optimD, criterion, dataloader, iteration, device,
              mnist_model_ref):
        real_label = 1
        fake_label = 0

        for i, data in enumerate(dataloader, 0):
            if i >= train_iterations_per_step:
                break

            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full(
                (b_size, ), real_label, dtype=torch.float, device=device)
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimD.step()

            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimG.step()

            is_score, is_std = inception_score(fake, mnist_model_ref)

            # Output training stats
           # if iteration % 10 == 0:
            #    print("[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z))"
             #         ": %.4f / %.4f \tInception score: %.4f" %
              #        (iteration, len(dataloader), errD.item(), errG.item(), D_x,
               #        D_G_z1, D_G_z2, is_score))

        return errG.item(), errD.item(), is_score


    def plot_images(dataloader):
        # Plot some training images
        if 1==0:
            real_batch = next(iter(dataloader))
            plt.figure(figsize=(8, 8))
            plt.axis("off")
            plt.title("Original Images")
            plt.imshow(
                np.transpose(
                    vutils.make_grid(real_batch[0][:64], padding=2,
                                     normalize=True).cpu(), (1, 2, 0)))

            plt.show()


    def demo_gan(checkpoint_paths):
        img_list = []
        fixed_noise = torch.randn(64, nz, 1, 1)
        for netG_path in checkpoint_paths:
            loadedG = Generator()
            loadedG.load_state_dict(torch.load(netG_path)["netGmodel"])
            with torch.no_grad():
                fake = loadedG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        fig = plt.figure(figsize=(8, 8))
        plt.axis("off")
        ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)]
               for i in img_list]
        ani = animation.ArtistAnimation(
            fig, ims, interval=1000, repeat_delay=1000, blit=True)
        ani.save("./generated" +str(SA)+".gif", writer="imagemagick", dpi=72)
        plt.show()




    # __Trainable_begin__
    class PytorchTrainable(tune.Trainable):
        def setup(self, config):
            use_cuda = config.get("use_gpu") and torch.cuda.is_available()
            self.device = torch.device("cuda" if use_cuda else "cpu")
            self.netD = Discriminator().to(self.device)
            self.netD.apply(weights_init)
            self.netG = Generator().to(self.device)
            self.netG.apply(weights_init)
            self.criterion = nn.BCELoss()    
            self.optimizerD =optim.Adam(
                self.netD.parameters(),
                lr=config.get("lr", 0.01),
                betas=(beta1, 0.999))
            self.optimizerG =optim.Adam(
                self.netG.parameters(),
                lr=config.get("lr", 0.01),
                betas=(beta1, 0.999))

            with FileLock(os.path.expanduser("~/.data.lock")):
                self.dataloader = get_data_loader()
            self.mnist_model_ref = c["mnist_model_ref"]

        def step(self):
            lossG, lossD, is_score = train(self.netD, self.netG, self.optimizerG,
                                           self.optimizerD, self.criterion,
                                           self.dataloader, self._iteration,
                                           self.device, self.mnist_model_ref)
            return {"lossg": lossG, "lossd": lossD, "is_score": 10- is_score}

        def save_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save({
                "netDmodel": self.netD.state_dict(),
                "netGmodel": self.netG.state_dict(),
                "optimD": self.optimizerD.state_dict(),
                "optimG": self.optimizerG.state_dict(),
            }, path)

            return checkpoint_dir

        def load_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            checkpoint = torch.load(path)
            self.netD.load_state_dict(checkpoint["netDmodel"])
            self.netG.load_state_dict(checkpoint["netGmodel"])
            self.optimizerD.load_state_dict(checkpoint["optimD"])
            self.optimizerG.load_state_dict(checkpoint["optimG"])

        def reset_config(self, new_config):
            if "netD_lr" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["lr"] = 10**-(new_config["netD_lr"])
            if "netG_lr" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["lr"] = 10**-(new_config["netG_lr"])
            if "netD_B" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["betas"] = (10**-(new_config["netD_B"]),0.999)
            if "netG_B" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["betas"] = (10**-(new_config["netG_B"]),0.999)
            if "weight_decay1" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay1"])
            if "weight_decay2" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay2"])
            self.config = new_config
            return True

        def _export_model(self, export_formats, export_dir):
            if export_formats == [ExportFormat.MODEL]:
                path = os.path.join(export_dir, "exported_models")
                torch.save({
                    "netDmodel": self.netD.state_dict(),
                    "netGmodel": self.netG.state_dict()
                }, path)
                return {ExportFormat.MODEL: path}
            else:
                raise ValueError("unexpected formats: " + str(export_formats))



    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    import urllib.request
    # Download a pre-trained MNIST model for inception score calculation.
    # This is a tiny model (<100kb).
    if not os.path.exists(MODEL_PATH):
        print("downloading model")
        os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
        urllib.request.urlretrieve(
            "https://github.com/ray-project/ray/raw/master/python/ray/tune/"
            "examples/pbt_dcgan_mnist/mnist_cnn.pt", MODEL_PATH)

    dataloader = get_data_loader()
    if not args.smoke_test:
        plot_images(dataloader)

    # load the pretrained mnist classification model for inception_score
    mnist_cnn = Net()
    mnist_cnn.load_state_dict(torch.load(MODEL_PATH))
    mnist_cnn.eval()
    mnist_model_ref = ray.put(mnist_cnn)

    # __tune_begin__
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=5,
        hyperparam_mutations={
            # distribution for resampling
            "netG_lr": lambda: np.random.uniform(1, 9),
            "netD_lr": lambda: np.random.uniform(1, 9),
            "netD_B": lambda: np.random.uniform(0, 2),
            "netD_B": lambda: np.random.uniform(0, 2),
            "weight_decay1":lambda: np.random.uniform(3, 8),
            "weight_decay2":lambda: np.random.uniform(3, 8)
        })
    c={"mnist_model_ref" : mnist_model_ref}


    experiment_metrics= dict(metric="is_score",
        mode="min")

 
    
    if(SA==0):
        algo = HyperOptSearch(**experiment_metrics)
        name="hypher"

    if(SA==1):
        algo =   BayesOptSearch(**experiment_metrics) 
        name="Bayes"
        
    if(SA==2):
        algo = AxSearch(
            max_concurrent=2, #was working with 2
            **experiment_metrics
        )
        name="AX"

        
        
    if(SA==3):
        algo = NevergradSearch(
            optimizer=ng.optimizers.CMA,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG"
     
    if(SA==4):
        algo = NevergradSearch(
            optimizer=ng.optimizers.TwoPointsDE,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG2DE"

        
        
    if(SA==5):
        algo = NevergradSearch(
        optimizer=ng.optimizers.RandomSearch,**experiment_metrics
        # space=space,  # If you want to set the space manually
        )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="random"
                
    if(SA==6):
        dim_dict = {
        "netG_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2),
        "netD_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2)
        }
        algo = ZOOptSearch(
        algo="Asracos",  # only support Asracos currently
        #dim_dict=dim_dict,
      #  dim_dict=dim_dict,

        budget=100,
            **experiment_metrics
        #dim_dict=dim_dict,
       #     **zoopt_search_config,
        )
        name="zoo"


  #  if(SA!=4):
   #     scheduler = AsyncHyperBandScheduler(**experiment_metrics)
        
        
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

    tune_kwargs = {
    "num_samples": 32 if args.smoke_test else 32,
      "stop":{
            "training_iteration": 200,
        },  

    "config": {
     "lr1":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    , "lr2":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    ,     "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0    
    ,     "beta1":2 #tune.uniform(1, 5),#,1e-4), #*10 et 0
     ,    "beta2":1 #tune.uniform(1, 5),#,1e-4), #*10 et 0
      ,  "adam1":0#tune.uniform(0,1),
      ,  "adam2":0#tune.uniform(0,1),
        }
    }
    def trial_name_id(trial):
        return f"{trial.trainable_name}_{trial.trial_id}"

    analysis = tune.run(
    PytorchTrainable,
    name=name,
    scheduler=scheduler,
    reuse_actors=True,
    search_alg=algo,
    verbose=0,
    checkpoint_at_end=True,
    num_samples=64,
   # export_formats=[ExportFormat.MODEL],
        config =     {
            "netG_lr": tune.uniform(1, 9),
           "netD_lr": tune.uniform(1, 9),
            "netG_B": tune.uniform(0, 2),
           "netD_B": tune.uniform(0, 2),
            "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)
        },      stop={
            "training_iteration": 100,
        },        metric="is_score",
        mode="min",
                  loggers=[TestLogger])

    all_trials = analysis.trials
    checkpoint_paths = [
        os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
        for t in all_trials
    ] 
    demo_gan(checkpoint_paths)
    

In [2]:
ray.init()


2020-12-13 19:31:10,251	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.80',
 'raylet_ip_address': '192.168.1.80',
 'redis_address': '192.168.1.80:6379',
 'object_store_address': '/tmp/ray/session_2020-12-13_19-31-09_702624_25077/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-13_19-31-09_702624_25077/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-13_19-31-09_702624_25077',
 'metrics_export_port': 47876,
 'node_id': '7f5634afb191308b679b8eb80871be932dd75604'}

In [4]:
from ray.tune.logger import *
supernombre="PB2TRAN";
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("/home/antoine/Projet/NovelTuning/", supernombre+"com.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        #if "done" in tmp:
         #   if(tmp["done"] != True):

        if "config" in tmp:
            del tmp["config"]
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            if not self._continuing:
                self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
            
GAN_MNIST(5)

    

Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2
PytorchTrainable_50c0e7c2,RUNNING,,1.55909,5.0446,1.04241,6.44886,5.49532,5.60377


(pid=11261)  /home/antoine/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2539: UserWarning:Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
(pid=11261)  /home/antoine/anaconda3/lib/python3.7/site-packages/ray/workers/default_worker.py:254: UserWarning:Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=11260)  /home/antoine/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2539: UserWarning:Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
(pid=11260)  /home/antoine/anaconda3/lib/python3.7/site-packages/ray/workers/default_worker.py:254: UserWarning:Implicit dimension choice for softma

Result for PytorchTrainable_50c91d8e:
  date: 2020-12-13_19-52-22
  done: false
  experiment_id: 62cf234d004d40bfa4d45d3deb9b7fcc
  experiment_tag: 2_netD_B=1.5949,netD_lr=5.7869,netG_B=1.1158,netG_lr=7.0348,weight_decay1=5.6784,weight_decay2=5.4397
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0062409244870973
  iterations_since_restore: 1
  lossd: 0.9530493021011353
  lossg: 2.8989431858062744
  node_ip: 192.168.1.80
  pid: 11261
  time_since_restore: 3.7344837188720703
  time_this_iter_s: 3.7344837188720703
  time_total_s: 3.7344837188720703
  timestamp: 1607885542
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 50c91d8e
  


Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_50c0e7c2,RUNNING,,1.55909,5.0446,1.04241,6.44886,5.49532,5.60377,,,,,
PytorchTrainable_50c91d8e,RUNNING,192.168.1.80:11261,1.5949,5.78686,1.11583,7.03477,5.67844,5.43969,1,3.73448,2.89894,0.953049,1.00624
PytorchTrainable_50cba07c,RUNNING,,1.22603,4.51579,0.956686,4.42174,4.05153,4.57097,,,,,
PytorchTrainable_50cde828,RUNNING,,1.40556,6.37432,0.802233,3.63806,4.62506,6.51921,,,,,
PytorchTrainable_50d0cb10,RUNNING,,1.42346,3.17885,0.733489,8.24188,5.19311,5.9478,,,,,
PytorchTrainable_50d47922,RUNNING,,1.28504,5.50596,1.20499,5.71786,5.51386,7.34351,,,,,
PytorchTrainable_50d75598,RUNNING,,0.818757,2.33412,0.710021,5.86935,4.87781,7.1403,,,,,
PytorchTrainable_50da1f3a,RUNNING,,1.21699,4.69192,0.95693,4.67412,6.23599,5.33591,,,,,


Result for PytorchTrainable_50cba07c:
  date: 2020-12-13_19-52-22
  done: false
  experiment_id: 21a16c207f094555a1f71eeaaacaa6da
  experiment_tag: 3_netD_B=1.226,netD_lr=4.5158,netG_B=0.95669,netG_lr=4.4217,weight_decay1=4.0515,weight_decay2=4.571
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0021954823129695
  iterations_since_restore: 1
  lossd: 2.193899631500244
  lossg: 6.722167491912842
  node_ip: 192.168.1.80
  pid: 11262
  time_since_restore: 3.7741875648498535
  time_this_iter_s: 3.7741875648498535
  time_total_s: 3.7741875648498535
  timestamp: 1607885542
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 50cba07c
  
Result for PytorchTrainable_50c0e7c2:
  date: 2020-12-13_19-52-22
  done: false
  experiment_id: cd4825d5fa194e0a943b89fc5e9d804e
  experiment_tag: 1_netD_B=1.5591,netD_lr=5.0446,netG_B=1.0424,netG_lr=6.4489,weight_decay1=5.4953,weight_decay2=5.6038
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0115722617561136
  iterat

...

Result for PytorchTrainable_75311acc:
  date: 2020-12-13_20-07-17
  done: false
  experiment_id: f287bbfcbdeb4b87b0c81871618bb0a0
  experiment_tag: 16_netD_B=0.96126,netD_lr=6.5436,netG_B=1.2794,netG_lr=2.6572,weight_decay1=5.5555,weight_decay2=4.8816@perturbed[netD_B=0.0,netD_lr=1.0,netG_lr=1.0,weight_decay1=8.0,weight_decay2=8.0]
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 4.920613157513151
  iterations_since_restore: 6
  lossd: 1.1899161338806152
  lossg: 0.6280682682991028
  node_ip: 192.168.1.80
  pid: 19510
  time_since_restore: 19.751450061798096
  time_this_iter_s: 2.1674954891204834
  time_total_s: 305.7618839740753
  timestamp: 1607886437
  timesteps_since_restore: 0
  training_iteration: 96
  trial_id: 75311acc
  
Result for PytorchTrainable_726e0b7e:
  date: 2020-12-13_20-07-17
  done: false
  experiment_id: f287bbfcbdeb4b87b0c81871618bb0a0
  experiment_tag: 14_netD_B=1.1224,netD_lr=6.3481,netG_B=1.3193,netG_lr=3.1266,weight_decay1=5.1586,weight_decay2=5.3537

Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_69071c88,RUNNING,192.168.1.80:19111,2,9,0.834462,1,3,8,99,320.053,1.33573,0.902961,5.41726
PytorchTrainable_727cb08e,RUNNING,192.168.1.80:19362,0,6.49058,0.834462,1,3,3,99,323.048,1.95044,1.17074,4.72213
PytorchTrainable_75311acc,RUNNING,192.168.1.80:19510,0,1,0.834462,1,8,8,97,308.029,1.22664,1.08248,4.81369
PytorchTrainable_50c0e7c2,TERMINATED,,0,9,0.733489,9,3,8,100,413.769,1.56664,1.00933,4.7501
PytorchTrainable_50cba07c,TERMINATED,,0,1,0.733489,9,8,3,100,417.318,0.920683,1.50723,3.96988
PytorchTrainable_50cde828,TERMINATED,,2,1,0.733489,9,8,4.03629,100,414.353,1.81509,1.24731,4.90466
PytorchTrainable_50d0cb10,TERMINATED,,0,1,0.733489,1,3,3,100,419.116,1.46058,1.22338,4.23552
PytorchTrainable_50d47922,TERMINATED,,0,9,0.733489,1,3,8,100,418.29,1.0491,1.16856,4.09731
PytorchTrainable_50d75598,TERMINATED,,0.818757,2.33412,0.710021,5.86935,4.87781,7.1403,100,413.69,1.75562,1.22149,5.15322
PytorchTrainable_50da1f3a,TERMINATED,,0,9,0.733489,1,3,8,100,416.385,1.47583,1.14314,4.27317


Result for PytorchTrainable_727cb08e:
  date: 2020-12-13_20-07-22
  done: true
  experiment_id: f287bbfcbdeb4b87b0c81871618bb0a0
  experiment_tag: 15_netD_B=0.95143,netD_lr=4.5373,netG_B=1.1086,netG_lr=5.6063,weight_decay1=6.7946,weight_decay2=5.735@perturbed[netD_B=0.0,netD_lr=6.49058,netG_lr=1.0,weight_decay1=3.0,weight_decay2=3.0]
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 4.7968709356450585
  iterations_since_restore: 10
  lossd: 1.067521333694458
  lossg: 1.470740795135498
  node_ip: 192.168.1.80
  pid: 19362
  time_since_restore: 35.13453221321106
  time_this_iter_s: 2.744107961654663
  time_total_s: 325.79177594184875
  timestamp: 1607886442
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: 727cb08e
  
Result for PytorchTrainable_75311acc:
  date: 2020-12-13_20-07-22
  done: false
  experiment_id: f287bbfcbdeb4b87b0c81871618bb0a0
  experiment_tag: 16_netD_B=0.96126,netD_lr=6.5436,netG_B=1.2794,netG_lr=2.6572,weight_decay1=5.5555,weight_decay2=4.8

Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_50c0e7c2,TERMINATED,,0,9,0.733489,9,3,8,100,413.769,1.56664,1.00933,4.7501
PytorchTrainable_50cba07c,TERMINATED,,0,1,0.733489,9,8,3,100,417.318,0.920683,1.50723,3.96988
PytorchTrainable_50cde828,TERMINATED,,2,1,0.733489,9,8,4.03629,100,414.353,1.81509,1.24731,4.90466
PytorchTrainable_50d0cb10,TERMINATED,,0,1,0.733489,1,3,3,100,419.116,1.46058,1.22338,4.23552
PytorchTrainable_50d47922,TERMINATED,,0,9,0.733489,1,3,8,100,418.29,1.0491,1.16856,4.09731
PytorchTrainable_50d75598,TERMINATED,,0.818757,2.33412,0.710021,5.86935,4.87781,7.1403,100,413.69,1.75562,1.22149,5.15322
PytorchTrainable_50da1f3a,TERMINATED,,0,9,0.733489,1,3,8,100,416.385,1.47583,1.14314,4.27317
PytorchTrainable_6c2b2982,TERMINATED,,0,1,0.733489,9,3,3,100,418.846,1.21952,1.1428,4.22835
PytorchTrainable_53436de8,TERMINATED,,0,9,0.733489,9,8,8,100,416.874,2.14116,1.35255,4.75572
PytorchTrainable_61918808,TERMINATED,,0,9,0.733489,1,8,8,100,417.317,1.17146,1.33568,4.23789


TuneError: ('Trials did not complete', [PytorchTrainable_50c91d8e])

In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

def train_VAE(config):
    parser = argparse.ArgumentParser(description='VAE MNIST Example')
    parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 128)')
    parser.add_argument('--epochs', type=int, default=training, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args, _ = parser.parse_known_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if args.cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
    #train_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=True, download=True,
    #                   transform=transforms.ToTensor()),
    #    batch_size=args.batch_size, shuffle=True, **kwargs)
    #test_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    #batch_size=args.batch_size, shuffle=True, **kwargs)



    class VAE(nn.Module):
        def __init__(self):
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            super(VAE, self).__init__()

            dim1= int(config.get("d1", 0)) #36*12*4*3
            dim2=int(config.get("d2", 0)) #36*12*3
            dim3=int(config.get("d3", 0)) #20*27
            dim4=int(config.get("d4", 0)) #dim2
            dim5=int(config.get("d5", 0)) #dim1


           # self.fc0 = nn.Linear(32*32*3,400*3)
          #  self.fc1 = nn.Linear(784*3, 400*3)
          #  self.fc21 = nn.Linear(400*3, 20*3)
          #  self.fc22 = nn.Linear(400*3, 20*3)
          #  self.fc3 = nn.Linear(20*3, 400*3)
           # self.fc4 = nn.Linear(400*3, 784*3)
           # self.fc5 = nn.Linear(400*3,32*32*3)
            
            self.lf=cst*cst*3

            self.fc1 = nn.Linear(self.lf, dim1)

            self.fc12 = nn.Linear(dim1, dim2)

            self.fc21 = nn.Linear(dim2, dim3)
            self.fc22 = nn.Linear(dim2, dim3)

            self.fc23 = nn.Linear(dim3,dim4)

            self.fc3 = nn.Linear(dim4, dim5)

            self.fc4 = nn.Linear(dim5, self.lf)

        def encoder(self, x):
            h1 = F.relu(self.fc1(x.view(-1, 32*32*3)))
            h1 = F.relu(self.fc12(h1))
            return self.fc21(h1), self.fc22(h1)

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return mu + eps*std

        def decoder(self, z):
            h3 = F.relu(self.fc23(z))
            h3 = F.relu(self.fc3(h3))
            return torch.sigmoid(self.fc4(h3))

        def forward(self, x):
            mu, logvar = self.encoder(x)
            z = self.reparameterize(mu, logvar)
            return self.decoder(z), mu, logvar


    model = VAE().to(device)
    #model = VAES(channel_in = 3).to(device)
    #optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)
    lr=10**-(config.get("lr", 0))
    beta1=1-10**-(config.get("b1", 0))
    beta2=1-10**-(config.get("b2", 0))
    eps=10**-(config.get("eps", 0))
    weight_decay=10**-(config.get("wd", 0))
    optimizer = optim.Adam(model.parameters()#, lr=1e-3)
                            ,  lr=lr, betas=(beta1, beta2),
                           eps=eps, weight_decay=weight_decay)
    #print(optimizer)
    
    
    
    
    # Reconstruction + KL divergence losses summed over all elements and batch
    def loss_function(recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, cst*cst*3), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD


    def train(train_loader):
        model.train()
        train_loss = 0
        model.to(device)
        for i in range(1):
          train_loss = 0
          for batch_idx, (data, _) in enumerate(train_loader):
        #     if(batch_idx>10):
          #        break
              data = data.to(device)
              optimizer.zero_grad()
              recon_batch, mu, logvar = model(data)
              loss = loss_function(recon_batch, data, mu, logvar)
              loss.backward()
              train_loss += loss.item()
              optimizer.step()
        print(len(train_loader.dataset))
        print(train_loss / len(train_loader.dataset))


    def test(test_loader):
        model.eval()
        test_loss = 0
        model.to(device)
        with torch.no_grad():
            for i, (data, _) in enumerate(test_loader):
           #     if(i>10):
            #        break
                data = data.to(device)
                recon_batch, mu, logvar = model(data)
                test_loss += loss_function(recon_batch, data, mu, logvar).item()
                if i == 0:
                    n = min(data.size(0), 8)
                    comparison = torch.cat([data[:n],
                                          recon_batch.view(batchSize, 3, cst, cst)[:n]])
                    save_image(comparison.cpu(),
                             path+ 'reconstruction_' + str(os.getpid()) + '.png', nrow=n)

        test_loss /= len(test_loader.dataset)
        tune.report(loss=test_loss)
        #print('====> Test set loss: {:.4f}'.format(test_loss))

        


    if __name__ == "__main__":
        train_loader=  get_pinned_object(X_id)
        test_loader=  get_pinned_object(Y_id)

        #test_loader = train_loader #get_pinned_object(Y_id)
        for epoch in range(1, args.epochs + 1):
            train(train_loader)
            test(test_loader)
            with torch.no_grad():
                sample = torch.randn(64,512).to(device)
                sample = model.decoder(sample).cpu()
                save_image(sample.view(64, 3, cst, cst),path +'sample_' + str(os.getpid()) + '.png')
        del train_loader
        del test_loader
    
    




from ray.tune.suggest import ConcurrencyLimiter

algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

algo = ConcurrencyLimiter(algo, max_concurrent=2)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = ASHAScheduler()


imageSize = 32

stopper = CustomStopper()
training=10
cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune
analysis = tune.run(
train_VAE,
name=name,
scheduler=scheduler,
reuse_actors=False,
search_alg=algo,
verbose=2,
checkpoint_at_end=False,
num_samples=1,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(2.5, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 4)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
# ,    "d1": tune.uniform(total/4, total)
# ,    "d2": tune.uniform(total/16, total/4)
# ,    "d3": tune.uniform(total/64, total/16)
# ,    "d4": tune.uniform(total/16, total/4)
#,    "d5": tune.uniform(total/4, total)

                
    },      stop=stopper
        ,        metric="loss",
    mode="min", resources_per_trial={'gpu': 1}

 #             loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths)
